# Python Libraries

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn
from sklearn import preprocessing
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, accuracy_score
from sklearn.feature_selection import SelectKBest, f_regression, RFE, RFECV, mutual_info_regression
from sklearn.svm import SVR
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import cross_validate
import seaborn as sns

# Data Preprocessing

In [ ]:
class Data_preprocessing:
    def __init__(self, train_filepath, test_filepath):
        self.train_filepath = train_filepath
        self.test_filepath = test_filepath
        
    def load_dataset(self):
        train_df = pd.read_csv(self.train_filepath)
        test_df = pd.read_csv(self.test_filepath)
        return train_df, test_df
    
    def extract_labels(self):
        train_df, test_df = self.load_dataset()
        columns = list(train_df.columns)
        return columns[len(columns)-3:]
    
    def categorical_to_numeric(self):
        train_df, test_df = self.load_dataset()
        categorical_features = train_df.select_dtypes(include=['object']).columns.tolist()
        binary_features = []
        
        for i in categorical_features:
            if len(train_df[i].unique()) == 2:
                categories = train_df[i].unique().tolist()
                train_df[i].replace({categories[0]: 0, categories[1]: 1}, inplace=True)
                test_df[i].replace({categories[0]: 0, categories[1]: 1}, inplace=True)
                
        # one hot encoding other nominal data
        nominal_features = train_df.select_dtypes(include=['object']).columns.tolist()
        tr_df = train_df.drop(nominal_features, axis=1)
        ts_df = test_df.drop(nominal_features, axis=1)
        encoder = preprocessing.OneHotEncoder(sparse=False)
        encoder.fit(train_df[nominal_features])
        encoder.categories_
        encoder.transform(train_df[nominal_features])
        train_df = pd.concat([train_df, pd.DataFrame(encoder.transform(train_df[nominal_features]))], axis=1)
        
        encoder.fit(test_df[nominal_features])
        encoder.categories_
        encoder.transform(test_df[nominal_features])
        test_df = pd.concat([test_df, pd.DataFrame(encoder.transform(test_df[nominal_features]))], axis=1)
        
        return train_df, test_df, nominal_features, tr_df, ts_df
    
    def train_test_sets(self, prior):
        train_df, test_df, nominal_features, tr_df, ts_df = self.categorical_to_numeric()
        train_df.drop(nominal_features, axis=1, inplace=True)
        test_df.drop(nominal_features, axis=1, inplace=True)
        
        train_labels = np.array(train_df[grades])
        train_df = train_df.drop(grades, axis=1)
        
        test_labels = np.array(test_df[grades])
        test_df = test_df.drop(grades, axis=1)
        
        if prior==True:
#             train_set = np.array(train_df.drop(grades[2], axis=1))
            train_labels = np.array(train_df[grades[2]])
#             test_set = np.array(test_df.drop(grades[2], axis=1))
            test_labels = np.array(test_df[grades[2]])
        
        return train_labels, test_labels, train_df, test_df

In [ ]:
train_path = "student_performance_train.csv"
test_path = "student_performance_test.csv"

preprocessor = Data_preprocessing(train_path, test_path)
og_train_df, og_test_df = preprocessor.load_dataset()
grades = preprocessor.extract_labels()
train_data_df, test_data_df, nom_feats, tr_df, ts_df = preprocessor.categorical_to_numeric()
labels, test_labels, train_df, test_df = preprocessor.train_test_sets(prior=False)

In [ ]:
ts_df = ts_df.drop(grades, axis=1)
tr_df = tr_df.drop(grades, axis=1)


# Baseline Models

In [ ]:
# Predict the first period academic performance
class Trivial_Regressor:
    def __init__(self, x, y):
        self.x = x
        self.y = y
        
    def predict(self, x_test):
        pred = np.zeros((x_test).shape[0])
        for i in range((x_test).shape[0]):
            pred[i] = np.mean(self.y)
        return pred
    
class OneNN:
    def __init__(self, x, y):
        self.x = x
        self.y = y
        
    def predict(self, test_point):
        euclidean_dist = np.zeros((self.x).shape[0])
        for i in range(len(self.x)):
#             print("sum", np.sqrt(np.sum((self.x[i, :] - test_point)**2)))
            euclidean_dist[i] = np.sqrt(np.sum((self.x.iloc[i, :] - test_point)**2))
#         print(euclidean_dist.shape)
        neighbors = np.argsort(euclidean_dist) # calculating the closest neighbors
        return np.mean(self.y[neighbors[:1]])

# Performance Measures
class error_estimation:
    def __init__(self, y_true):
        self.y_true = y_true

    ## Root Mean square Error
    def RMSE(self, y_pred):
        return mean_squared_error(self.y_true, y_pred, squared=False)

    ## Mean absolute error
    def MAE(self, y_pred):
        return mean_absolute_error(self.y_true, y_pred)

    ## R-squared value
    def r2(self, y_pred):
        return r2_score(self.y_true, y_pred)

In [ ]:
def plot_pred_dist(y_true, y_pred, m):   
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(9,5))
    ax1.hist(y_true, edgecolor='white')
    ax1.set_title(f'Actual G1 Distribution')
    ax2.hist(y_pred, edgecolor='white')
    ax2.set_title(f'Predicted G1 Distribution')
#     fig.suptitle(f'MISSION{m}')
#     plt.savefig(f'Images/Results/M{m}_dist')

## MISSION 1 
Predict first-period academic performance without any prior academic performance data

In [ ]:
##### TRIVIAL REGRESSION ######

regressor1 = Trivial_Regressor(tr_df, labels[:,0])
y_pred1 = regressor1.predict(ts_df)
error = error_estimation(test_labels[:, 0])
print(f'******************* For Mission 1 *******************')
print(f'--------- For Trivial Regressor ---------')
print(f'RMSE: {error.RMSE(y_pred1)}')
print(f'MAE: {error.MAE(y_pred1)}')
print(f'R-squared: {error.r2(y_pred1)}')

##### 1NN REGRESSION ######

regressor2 = OneNN(tr_df, labels[:, 0])
y_pred2 = np.zeros(test_labels.shape[0])
for i in range(ts_df.shape[0]):
    y_pred2[i] = regressor2.predict(np.array(ts_df)[i, :])
print(f'--------- For 1NN Regressor ---------')
print(f'RMSE: {error.RMSE(y_pred2)}')
print(f'MAE: {error.MAE(y_pred2)}')
print(f'R-squared: {error.r2(y_pred2)}')

##### Linear Regression ######

regressor3 = LinearRegression().fit(tr_df, labels[:, 0])
y_pred3 = regressor3.predict(ts_df)
print(f'--------- For Linear Regressor ---------')
print(f'RMSE: {error.RMSE(y_pred3)}')
print(f'MAE: {error.MAE(y_pred3)}')
print(f'R-squared: {error.r2(y_pred3)}')

In [ ]:
plot_pred_dist(labels[:,0], y_pred1, 1)
plt.savefig(f'Images/Results/M1_trivial_dist.png')

plot_pred_dist(labels[:,2], y_pred2, 2)
plt.savefig(f'Images/Results/M1_1nn_dist.png')

plot_pred_dist(labels[:,2], y_pred3, 3)
plt.savefig(f'Images/Results/M1_linear_dist.png')

## MISSION 2
Predict final-period academic performance without any prior academic performance data

In [ ]:
##### TRIVIAL REGRESSION ######

regressor1 = Trivial_Regressor(tr_df, labels[:,0])
y_pred1 = regressor1.predict(ts_df)
error = error_estimation(test_labels[:, 2])
print(f'******************* For Mission 2 *******************')
print(f'--------- For Trivial Regressor ---------')
print(f'RMSE: {error.RMSE(y_pred1)}')
print(f'MAE: {error.MAE(y_pred1)}')
print(f'R-squared: {error.r2(y_pred1)}')

##### 1NN REGRESSION ######

regressor2 = OneNN(tr_df, labels[:, 2])
# regressor2 = KNeighborsRegressor(n_neighbors=1).fit(tr_df, labels[:,2])
# y_pred2 = regressor2.predict(ts_df)
y_pred2 = np.zeros(test_labels.shape[0])
for i in range(ts_df.shape[0]):
    y_pred2[i] = regressor2.predict(np.array(ts_df)[i, :])
    
print(f'--------- For 1NN Regressor ---------')
print(f'RMSE: {error.RMSE(y_pred2)}')
print(f'MAE: {error.MAE(y_pred2)}')
print(f'R-squared: {error.r2(y_pred2)}')

##### Linear Regression ######

regressor3 = LinearRegression().fit(tr_df, labels[:, 2])
y_pred3 = regressor3.predict(ts_df)
print(f'--------- For Linear Regressor ---------')
print(f'RMSE: {error.RMSE(y_pred3)}')
print(f'MAE: {error.MAE(y_pred3)}')
print(f'R-squared: {error.r2(y_pred3)}')

In [ ]:
plot_pred_dist(labels[:,0], y_pred1, 1)
plt.savefig(f'Images/Results/M2_trivial_dist.png')

plot_pred_dist(labels[:,2], y_pred2, 2)
plt.savefig(f'Images/Results/M2_1nn_dist.png')

plot_pred_dist(labels[:,2], y_pred3, 3)
plt.savefig(f'Images/Results/M2_linear_dist.png')

## MISSION 3
Predict final academic performance using all available prior academic performance data

In [ ]:
tr_df['G1'] = labels[:,0]
tr_df['G2'] = labels[:,1]

ts_df['G1'] = test_labels[:,0]
ts_df['G2'] = test_labels[:,1]

In [ ]:
##### TRIVIAL REGRESSION ######

regressor1 = Trivial_Regressor(tr_df, labels[:,2])
y_pred1 = regressor1.predict(ts_df)
error = error_estimation(test_labels[:,2])
print(f'******************* For Mission 3 *******************')
print(f'--------- For Trivial Regressor ---------')
print(f'RMSE: {error.RMSE(y_pred1)}')
print(f'MAE: {error.MAE(y_pred1)}')
print(f'R-squared: {error.r2(y_pred1)}')

##### 1NN REGRESSION ######

regressor2 = OneNN(tr_df, labels[:,2])
y_pred2 = np.zeros(test_labels.shape[0])
for i in range(ts_df.shape[0]):
    y_pred2[i] = regressor2.predict(np.array(ts_df)[i, :])
print(f'--------- For 1NN Regressor ---------')
print(f'RMSE: {error.RMSE(y_pred2)}')
print(f'MAE: {error.MAE(y_pred2)}')
print(f'R-squared: {error.r2(y_pred2)}')

##### Linear Regression ######

regressor3 = LinearRegression().fit(tr_df, labels[:,2])
y_pred3 = regressor3.predict(ts_df)
print(f'--------- For Linear Regressor ---------')
print(f'RMSE: {error.RMSE(y_pred3)}')
print(f'MAE: {error.MAE(y_pred3)}')
print(f'R-squared: {error.r2(y_pred3)}')

In [ ]:
plot_pred_dist(labels[:,0], y_pred1, 1)
plt.savefig(f'Images/Results/M3_trivial_dist.png')

plot_pred_dist(labels[:,2], y_pred2, 2)
plt.savefig(f'Images/Results/M3_1nn_dist.png')

plot_pred_dist(labels[:,2], y_pred3, 3)
plt.savefig(f'Images/Results/M3_linear_dist.png')

# DATA VISUALIZATIONS

## Exploratory Data Analysis

### Histograms, Barplots, pieplots, KDE plots

In [ ]:
# Distribution of Final Grades - G1, G2, G3
def grades_distribution(df, i):
    plt.figure()
    cmap = plt.get_cmap("tab20c")
    plt.hist(df[i], color=cmap(1), edgecolor='black')
    plt.xlabel('Grade')
    plt.ylabel('Count')
    plt.title('Distribution of Final Grades')
    plt.savefig(f'Images/Feature Selection/{i}_histogram.png')

In [ ]:
grades_distribution(og_train_df, 'G1')
grades_distribution(og_train_df, 'G2')
grades_distribution(og_train_df, 'G3')

In [ ]:
fig, axes = plt.subplots(1,3, figsize=(15,5), sharey=True)
for i in range(1, 4):
    sns.kdeplot(data=og_train_df[grades].iloc[:,i-1], thresh=0.4, ax=axes[i-1])
plt.suptitle(f'KDE plots of all the Grades')
plt.savefig(f'Images/Feature Selection/KDE_grades.png')

In [ ]:
categ_feat = og_train_df.select_dtypes(include=['object']).columns.tolist()
feat_df = og_train_df.drop(grades, axis=1)
fig, ax = plt.subplots(6, 5, figsize=(15,11))
fig.tight_layout()
cmap = plt.get_cmap("tab20c")
colors = cmap(np.array([1, 5, 6, 9, 11]))
for i, feat in enumerate(feat_df.columns):
    if feat in categ_feat:
        plt.sca(fig.axes[i])
        feat_df[feat].value_counts().plot.pie(colors=colors)
        fig.axes[i].axis('off')
    else:
        fig.axes[i].hist(feat_df[feat], color=cmap(1), edgecolor='black')
    fig.axes[i].set_title(feat)
plt.savefig(f'Images/Feature Selection/features_info.png')

### Readjusting the dataframes for better handling

In [ ]:
def train_test(train_df, test_df):
    grades1 = train_df['G1']
    grades2 = train_df['G2']
    grades3 = train_df['G3']
    train_df = train_df.drop(grades, axis=1)
    train_df.loc[:,'G1'] = grades1
    train_df.loc[:,'G2'] = grades2
    train_df.loc[:,'G3'] = grades3
    
    grades1 = test_df['G1']
    grades2 = test_df['G2']
    grades3 = test_df['G3']
    test_df = test_df.drop(grades, axis=1)
    test_df.loc[:,'G1'] = grades1
    test_df.loc[:,'G2'] = grades2
    test_df.loc[:,'G3'] = grades3
    
    return train_df, test_df

In [ ]:
new_train_df, new_test_df = train_test(train_data_df, test_data_df)

## Feature Selection

### 1. Pearson Correlation Method

In [ ]:
## Correlation Matrix
def correlation_matrix(df, m):
    fig, axes = plt.subplots(figsize = (20, 17))
    feature_corr = df.corr()
    sns.heatmap(feature_corr, mask=np.zeros_like(feature_corr),
                cmap=sns.diverging_palette(220, 10, as_cmap=True),
                square=True, ax=axes)
    plt.title(f'Confusion Matrix for Mission{m+1}')
    plt.savefig(f'Images/Feature Selection/correlation_matrix_{m}.png')
    return feature_corr

In [ ]:
## remove one of the two features whose correlation coefficient value is greater then 0.85
def get_correlated_columns(df, c):
    cols_corr = set() # set of all columns that are correlated
    corr_matrix = df.corr()
    for row in range(len(corr_matrix.columns)):
        if abs(corr_matrix.iloc[row,-1]) < c:
            cols_name = corr_matrix.columns[row] # get the column name
            cols_corr.add(cols_name)
    return cols_corr

In [ ]:
new_train_df = new_train_df.drop(nom_feats, axis=1)
new_test_df = new_test_df.drop(nom_feats, axis=1)

In [ ]:
train_df1 = new_train_df.drop(['G2', 'G3'], axis=1)
test_df1 = new_test_df.drop(['G2', 'G3'], axis=1)

train_df2 = new_train_df.drop(['G1', 'G2'], axis=1)
test_df2 = new_test_df.drop(['G1', 'G2'], axis=1)

train_df3 = new_train_df
test_df3 = new_test_df

In [ ]:
# Mission1 - remove G2, G3 and predict G1
X1 = train_df
testX1 = test_df
y1 = new_train_df["G1"]
testy1 = new_test_df["G1"]
features1 = X1.columns

# Mission2 - remove G1, G2 and predict G3
X2 = X1
testX2 = test_df
y2 = new_train_df["G3"]
testy2 = new_test_df["G3"]
features2 = X2.columns

# Mission3 - set G1, G2 as features and predict G3
X3 = new_train_df.drop(['G3'], axis=1)
testX3 = new_test_df.drop(['G3'], axis=1)
y3 = new_train_df["G3"]
testy3 = new_test_df["G3"]
features3 = X3.columns

In [ ]:
X_pearson = [train_df1, train_df2, train_df3]
testX_pearson = [test_df1, test_df2, test_df3]
threshold = 0.1
sf_p = []
for i in range(len(X_pearson)): 
    correlation_matrix(X_pearson[i], i)
    correlated_features = get_correlated_columns(X_pearson[i], threshold)
    print(f'Number of correlated features: {len(set(correlated_features))}')
    print(f'correlated features for X{i+1}: {correlated_features}')
    X_pearson[i] = X_pearson[i].drop(correlated_features, axis=1)
    testX_pearson[i] = testX_pearson[i].drop(correlated_features, axis=1)
    sf_p.append(X_pearson[i].columns)
    
X1_fs = X_pearson[0]
X2_fs = X_pearson[1]
X3_fs = X_pearson[2]

testX1_fs = testX_pearson[0]
testX2_fs = testX_pearson[1]
testX3_fs = testX_pearson[2]

### 2. Univariate Feature Selection -Fscore

In [ ]:
def univariate_fregr(x1, x2, x3, t1, t2, t3, Y1, Y2, Y3, k):
    sf_f = []
    f_selector = SelectKBest(score_func=f_regression, k=k)
    f_selector1 = f_selector.fit(np.array(x1), Y1)
    x1_f = f_selector1.transform(x1)
    t1_f = f_selector1.transform(t1)
    
    selected_features1 = [features1[i] for i in f_selector1.scores_.argsort()[::-1][:k]]
    sf_f.append(selected_features1)
    
    # Plot the scores
    plt.figure()
    plt.bar([i for i in range(len(f_selector1.scores_))], f_selector1.scores_)
    plt.xlabel(f'feature index')
    plt.ylabel(f'F-value')
    plt.title(f'F-score Plots')
    plt.savefig(f'Images/Feature Selection/F-scoreplots_1')
    
    f_selector2 = f_selector.fit(np.array(x2), Y2)
    x2_f = f_selector2.transform(x2)
    t2_f = f_selector2.transform(t2)
    
    selected_features2 = [features2[i] for i in f_selector2.scores_.argsort()[::-1][:k]]
    sf_f.append(selected_features2)
    
    # Plot the scores
    plt.figure()
    plt.bar([i for i in range(len(f_selector2.scores_))], f_selector2.scores_)
    plt.xlabel(f'feature index')
    plt.ylabel(f'F-value')
    plt.title(f'F-score Plots')
    plt.savefig(f'Images/Feature Selection/F-scoreplots_2')
    
    f_selector3 = f_selector.fit(np.array(x3), Y3)
    x3_f = f_selector3.transform(x3)
    t3_f = f_selector3.transform(t3)
    
    selected_features3 = [features3[i] for i in f_selector3.scores_.argsort()[::-1][:k]]
    sf_f.append(selected_features3)
    
    # Plot the scores
    plt.figure()
    plt.bar([i for i in range(len(f_selector3.scores_))], f_selector3.scores_)
    plt.xlabel(f'feature index')
    plt.ylabel(f'F-value')
    plt.title(f'F-score Plots')
    plt.savefig(f'Images/Feature Selection/F-scoreplots_3')
    
    return x1_f, t1_f, x2_f, t2_f, x3_f, t3_f, sf_f

In [ ]:
X1_f, testX1_f, X2_f, testX2_f, X3_f, testX3_f, selected_features_f = univariate_fregr(X1, X2, X3, 
                                                                    testX1, testX2, testX3, 
                                                                    y1, y2, y3, 25)

### 3. Recursive Feature Selection

In [ ]:
def recursive_FS(model, x1, x2, x3, t1, t2, t3, Y1, Y2, Y3, feat1, feat3):
    sf_rfe = []
    rfe1 = RFE(estimator=model, step=1)
    rfe1 = rfe1.fit(x1, Y1)
    sf1 = pd.DataFrame({'Feature': list(feat1),
                        'Ranking': rfe1.ranking_})
    sf1 = sf1.sort_values(by='Ranking')
    print(f'score for the transformed feature space: {rfe1.score(x1, Y1)}')
    x1_rfe = rfe1.transform(x1)
    t1_rfe = rfe1.transform(t1)
    sf_rfe.append(list(sf1['Feature']))
    
    rfe2 = RFE(estimator=model, step=1)
    rfe2 = rfe2.fit(x2, Y2)
    sf2 = pd.DataFrame({'Feature': list(feat1),
                        'Ranking': rfe2.ranking_})
    sf2 = sf2.sort_values(by='Ranking')
    print(f'score for the transformed feature space: {rfe2.score(x2, Y2)}')
    x2_rfe = rfe2.transform(x2)
    t2_rfe = rfe2.transform(t2)
    sf_rfe.append(list(sf2['Feature']))
    
    rfe3 = RFE(estimator=model, step=1)
    rfe3 = rfe3.fit(x3, Y3)
    sf3 = pd.DataFrame({'Feature': list(feat3),
                        'Ranking': rfe3.ranking_})
    sf3 = sf3.sort_values(by='Ranking')
    print(f'score for the transformed feature space: {rfe3.score(x3, Y3)}')
    x3_rfe = rfe3.transform(x3)
    t3_rfe = rfe3.transform(t3)
    sf_rfe.append(list(sf3['Feature']))
    
    return x1_rfe, t1_rfe, x2_rfe, t2_rfe, x3_rfe, t3_rfe, sf_rfe

In [ ]:
linregr = LinearRegression()
X1_rfe, testX1_rfe, X2_rfe, testX2_rfe, X3_rfe, testX3_rfe, selected_features_rfe = recursive_FS(linregr, X1, X2, X3, 
                                                                          testX1, testX2, testX3, 
                                                                          y1, y2, y3, features1, features3)

### 4. Recursive Feature Elimination Cross Validated

In [ ]:
def recursive_FS_CV(model, x1, x2, x3, t1, t2, t3, Y1, Y2, Y3):
    sf_rfecv = []
    rfecv = RFECV(estimator=model, step=1, min_features_to_select=5, cv=12, scoring='r2')
    rfecv1 = rfecv.fit(x1, Y1)
    print(f'Optimal number of features: {rfecv1.n_features_}')
    print(f'Best features; {x1.columns[rfecv1.support_]}')
    x1_rfecv = rfecv1.transform(x1)
    t1_rfecv = rfecv1.transform(t1)
    plt.figure()
    plt.xlabel(f'Number of features selected')
    plt.ylabel(f'Cross validation score of number of selected features')
    plt.plot(range(1, len(rfecv1.grid_scores_) + 1), rfecv1.grid_scores_)
    plt.title(f'Feature Ranking')
    plt.savefig(f'Images/Feature Selection/RFECV_CVscore_mission1.png')
    plt.show()
    print(f'Score for RFECV1: {rfecv1.score(x1, Y1)}')
    selected_features1 = x1.columns[rfecv1.support_]
    sf_rfecv.append(selected_features1)
    
    rfecv2 = rfecv.fit(x2, Y2)
    print(f'Optimal number of features: {rfecv2.n_features_}')
    print(f'Best features; {x2.columns[rfecv2.support_]}')
    x2_rfecv = rfecv2.transform(x2)
    t2_rfecv = rfecv2.transform(t2)
    plt.figure()
    plt.xlabel(f'Number of features selected')
    plt.ylabel(f'Cross validation score of number of selected features')
    plt.plot(range(1, len(rfecv2.grid_scores_) + 1), rfecv2.grid_scores_)
    plt.title(f'Feature Ranking')
    plt.savefig(f'Images/Feature Selection/RFECV_CVscore_mission2.png')
    plt.show()
    print(f'Score for RFECV2: {rfecv2.score(x2, Y2)}')
    selected_features2 = x2.columns[rfecv2.support_]
    sf_rfecv.append(selected_features2)
    
    rfecv3 = rfecv.fit(x3, Y3)
    print(f'Optimal number of features: {rfecv3.n_features_}')
    print(f'Best features; {x3.columns[rfecv3.support_]}')
    x3_rfecv = rfecv3.transform(x3)
    t3_rfecv = rfecv3.transform(t3)
    plt.figure()
    plt.xlabel(f'Number of features selected')
    plt.ylabel(f'Cross validation score of number of selected features')
    plt.title(f'Feature Ranking')
    plt.plot(range(1, len(rfecv3.grid_scores_) + 1), rfecv3.grid_scores_)
    plt.savefig(f'Images/Feature Selection/RFECV_CVscore_mission3.png')
    plt.show()
    print(f'Score for RFECV3: {rfecv3.score(x3, Y3)}')
    selected_features3 = x3.columns[rfecv3.support_]
    sf_rfecv.append(selected_features3)
    
    return x1_rfecv, t1_rfecv, x2_rfecv, t2_rfecv, x3_rfecv, t3_rfecv, sf_rfecv

In [ ]:
X1_rfecv, testX1_rfecv, X2_rfecv, testX2_rfecv, X3_rfecv, testX3_rfecv, selected_features_rfecv = recursive_FS_CV(linregr, X1, X2, X3, 
                                                                                         testX1, testX2, testX3, 
                                                                                         y1, y2, y3)

### 5. Mutual information 

In [ ]:
def mutual_info(x1, x2, x3, t1, t2, t3, Y1, Y2, Y3, k):
    sf_mi = []
    f_selector = SelectKBest(score_func=mutual_info_regression, k=k)
    
    # fit the data to the selector
    f_selector1 = f_selector.fit(x1, Y1)
    
    # Transform
    x1_mi = f_selector1.transform(x1)
    
    # Transform test input data
    t1_mi = f_selector1.transform(t1)
    
    selected_features1 = [features1[i] for i in f_selector1.scores_.argsort()[::-1][:k]]
    sf_mi.append(selected_features1)
    
    plt.figure()
    # Plot scores
    plt.bar([i for i in range(len(f_selector1.scores_))], f_selector1.scores_)
    plt.xlabel(f'feature index')
    plt.ylabel(f'Estimated Mutual Index value')
    plt.title(f'Mutual Information')
    plt.savefig(f'Images/Feature Selection/mutual_info_1.png')
    
    # fit the data to the selector
    f_selector2 = f_selector.fit(x2, Y2)
    
    # Transform
    x2_mi = f_selector2.transform(x2)
    
    # Transform test input data
    t2_mi = f_selector2.transform(t2)
    
    selected_features2 = [features2[i] for i in f_selector2.scores_.argsort()[::-1][:k]]
    sf_mi.append(selected_features2)
    
    plt.figure()
    # Plot scores
    plt.bar([i for i in range(len(f_selector2.scores_))], f_selector2.scores_)
    plt.xlabel(f'feature index')
    plt.ylabel(f'Estimated Mutual Index value')
    plt.title(f'Mutual Information')
    plt.savefig(f'Images/Feature Selection/mutual_info_2.png')
    
    # fit the data to the selector
    f_selector3 = f_selector.fit(x3, Y3)
    
    # Transform
    x3_mi = f_selector3.transform(x3)
    
    # Transform test input data
    t3_mi = f_selector3.transform(t3)
    
    selected_features3 = [features3[i] for i in f_selector3.scores_.argsort()[::-1][:k]]
    sf_mi.append(selected_features3)
    
    plt.figure()
    # Plot scores
    plt.bar([i for i in range(len(f_selector3.scores_))], f_selector3.scores_)
    plt.xlabel(f'feature index')
    plt.ylabel(f'Estimated Mutual Index value')
    plt.title(f'Mutual Information')
    plt.savefig(f'Images/Feature Selection/mutual_info_3.png')
    
    sf = set(selected_features3).intersection(list(set(selected_features1).intersection(selected_features2)))
    
    return x1_mi, t1_mi, x2_mi, t2_mi, x3_mi, t3_mi, sf_mi
    

In [ ]:
X1_mi, testX1_mi, X2_mi, testX2_mi, X3_mi, testX3_mi, selected_features_mi = mutual_info(X1, X2, X3, 
                                                                   testX1, testX2, testX3, 
                                                                   y1, y2, y3, 25)


## MODEL SELECTION

In [ ]:
## Get the common features selected by all the models
def get_common_features(sf1, sf2, sf3, sf4, sf5):
    comm_feat = []
    for i in range(len(sf1)):
        intersection_list1 = set(sf1[i])
        intersection_list2 = set(intersection_list1.intersection(set(sf2[i])))                                     
        intersection_list3 = set(intersection_list2.intersection(set(sf3[i])))
        intersection_list4 = set(intersection_list3.intersection(set(sf4[i]))) 
        intersection_list5 = set(intersection_list4.intersection(set(sf5[i])))  
        comm_feat.append(intersection_list5)
    return comm_feat

In [ ]:
sf = get_common_features(sf_p, selected_features_f, selected_features_rfe, selected_features_rfecv, selected_features_mi)

In [ ]:
new_X1 = X1[sf[0]]
new_X2 = X2[sf[1]]
new_X3 = X3[sf[2]]

new_testX1 = testX1[sf[0]]
new_testX2 = testX2[sf[1]]
new_testX3 = testX3[sf[2]]

In [ ]:
# List to store R2_scores for each model and different Features Selections
SVR_R2, lr_R2, KNN_R2, NN_R2 = [], [], [], []

### Data Normalization

In [ ]:
def standardization(x, t, Y):
    y_mean = np.mean(Y)
    y_std = np.std(Y)
    for i in range(x.shape[1]):
        x_mean = np.mean(x[:,i])
        x_std = np.std(x[:,i])
        for j in range(x.shape[0]):
            x[j,i] = (x[j,i]-x_mean)/x_std
    

In [ ]:
def model_selection(model, x, Y, t, tY):
#     regr = make_pipeline(StandardScaler(), model)
    regr = make_pipeline(model)
    cv_regr = cross_validate(regr, np.array(x), Y, cv=5, return_train_score=True, return_estimator=True)
    val_scores = cv_regr['test_score']
    train_scores = cv_regr['train_score']
    optimal_estimator = cv_regr['estimator'][np.argmax(val_scores)]
    
    return np.mean(val_scores), np.mean(train_scores), optimal_estimator

In [ ]:
# plot the scores for different parameters
def plot_model_param(x, val_scores, tr_scores, m):
    plt.figure()
    plt.plot(x, val_scores, 'b', label='Val score')
    plt.plot(x, tr_scores, 'r', label='Train score')
    plt.title(f'r2 score plots - Mission_{m}')
    plt.legend()

In [ ]:
def test_model(estimator, x, y, t, tY):
    score_out = []
    test_regr = estimator
    test_regr.fit(x, y)
    y_pred = test_regr.predict(t)
    error = error_estimation(np.array(tY))
    print(f'RMSE: {error.RMSE(y_pred)}')
    score_out.append(error.RMSE(y_pred))
    print(f'MAE: {error.MAE(y_pred)}')
    score_out.append(error.MAE(y_pred))
    print(f'R2: {test_regr.score(t, tY)}') 
    score_out.append(test_regr.score(t, tY))
        
    return y_pred, score_out

In [ ]:
def plot_pred_dist(y_true, y_pred, m):   
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(9,5))
    ax1.hist(y_true, edgecolor='white')
    ax1.set_title(f'Actual G1 Distribution')
    ax2.hist(y_pred, edgecolor='white')
    ax2.set_title(f'Predicted G1 Distribution')
    fig.suptitle(f'MISSION{m}')

### 1. KNN Regression

In [ ]:
knn_SCORE1, knn_SCORE2, knn_SCORE3 = [], [], []
knn_tr_SCORE1, knn_tr_SCORE2, knn_tr_SCORE3 = [], [], []
ESTIMATOR1, ESTIMATOR2, ESTIMATOR3 = [], [], []

samples = np.arange(1, 30, 1)
for k in samples:
    knn_score1, train_score1, estimator1 = model_selection(KNeighborsRegressor(n_neighbors=k), X1_rfe, y1, testX1_rfe, testy1)
    knn_SCORE1.append(svr_score1)
    ESTIMATOR1.append(estimator1)
    knn_tr_SCORE1.append(train_score1)
    
    knn_score2, train_score2, estimator2 = model_selection(KNeighborsRegressor(n_neighbors=k), X2_rfe, y2, testX2_rfe, testy2)
    knn_SCORE2.append(svr_score2)
    ESTIMATOR2.append(estimator2)
    knn_tr_SCORE2.append(train_score2)
    
    knn_score3, train_score3, estimator3 = model_selection(KNeighborsRegressor(n_neighbors=k), X3_rfe, y3, testX3_rfe, testy3)
    knn_SCORE3.append(svr_score3)
    ESTIMATOR3.append(estimator3)
    knn_tr_SCORE3.append(train_score3)

optim_est1 = ESTIMATOR1[np.argmax(knn_SCORE1)]
plot_model_param(samples, knn_SCORE1, knn_tr_SCORE1, m=1)
optimal_param1 = samples[np.argmax(knn_SCORE1)]
print(f'optimal_epsilon1: {optimal_param1}')
print(f'optimal r2_score for Mission1: {np.max(knn_SCORE1)}')
plt.savefig(f'Images/Results/knn_mission1_scores.png')

optim_est2 = ESTIMATOR2[np.argmax(knn_SCORE2)]
plot_model_param(samples, knn_SCORE2, knn_tr_SCORE2, m=2)
optimal_param2 = samples[np.argmax(knn_SCORE2)]
print(f'optimal_epsilon2: {optimal_param2}')
print(f'optimal r2_score for Mission2: {np.max(knn_SCORE2)}')
plt.savefig(f'Images/Results/knn_mission2_scores.png')

optim_est3 = ESTIMATOR3[np.argmax(knn_SCORE3)]
plot_model_param(samples, knn_SCORE3, knn_tr_SCORE1, m=3)
optimal_param3 = samples[np.argmax(knn_SCORE3)]
print(f'optimal_epsilon1: {optimal_param3}')
print(f'optimal r2_score for Mission3: {np.max(knn_SCORE3)}')
plt.savefig(f'Images/Results/knn_mission3_scores.png')

In [ ]:
score_knn = []
print(f'Mission1')
y_pred1, score_out1 = test_model(optim_est1, X1_rfe, y1, testX1_rfe, np.array(testy1))
score_knn.append(score_out1)
print(f'Mission2')
y_pred2, score_out2 = test_model(optim_est2, X2_rfe, y2, testX2_rfe, np.array(testy2))
score_knn.append(score_out2)
print(f'Mission3')
y_pred3, score_out3 = test_model(optim_est3, X3_rfe, y3, testX3_rfe, np.array(testy3))
score_knn.append(score_out3)

In [ ]:
plot_pred_dist(y1, y_pred1, 1)
plt.savefig(f'Images/Results/knn_M1_dist.png')
plot_pred_dist(y2, y_pred2, 2)
plt.savefig(f'Images/Results/knn_M2_dist.png')
plot_pred_dist(y3, y_pred3, 3)
plt.savefig(f'Images/Results/knn_M3_dist.png')

### 2. Support Vector Regressor

In [ ]:
svr_SCORE1, svr_SCORE2, svr_SCORE3 = [], [], []
svr_tr_SCORE1, svr_tr_SCORE2, svr_tr_SCORE3 = [], [], []
ESTIMATOR1, ESTIMATOR2, ESTIMATOR3 = [], [], []

samples = np.arange(0.01, 50, 0.1)
for c in samples:
    svr_score1, train_score1, estimator1 = model_selection(SVR(C=c, epsilon=0.1), X1_rfe, y1, testX1_rfe, testy1)
    svr_SCORE1.append(svr_score1)
    ESTIMATOR1.append(estimator1)
    svr_tr_SCORE1.append(train_score1)
    svr_score2, train_score2, estimator2 = model_selection(SVR(C=c, epsilon=0.1), X2_rfe, y2, testX2_rfe, testy2)
    svr_SCORE2.append(svr_score2)
    ESTIMATOR2.append(estimator2)
    svr_tr_SCORE2.append(train_score2)
    svr_score3, train_score3, estimator3 = model_selection(SVR(C=c, epsilon=0.1), X3_rfe, y3, testX3_rfe, testy3)
    svr_SCORE3.append(svr_score3)
    ESTIMATOR3.append(estimator3)
    svr_tr_SCORE3.append(train_score3)

optim_est1 = ESTIMATOR1[np.argmax(svr_SCORE1)]
plot_model_param(samples, svr_SCORE1, svr_tr_SCORE1, m=1)
optimal_param1 = samples[np.argmax(svr_SCORE1)]
print(f'optimal_epsilon1: {optimal_param1}')
print(f'optimal r2_score for Mission1: {np.max(svr_SCORE1)}')
plt.savefig(f'Images/Results/svr_mission1_scores.png')

optim_est2 = ESTIMATOR2[np.argmax(svr_SCORE2)]
plot_model_param(samples, svr_SCORE2, svr_tr_SCORE2, m=2)
optimal_param2 = samples[np.argmax(svr_SCORE2)]
print(f'optimal_epsilon2: {optimal_param2}')
print(f'optimal r2_score for Mission2: {np.max(svr_SCORE2)}')
plt.savefig(f'Images/Results/svr_mission2_scores.png')

optim_est3 = ESTIMATOR3[np.argmax(svr_SCORE3)]
plot_model_param(samples, svr_SCORE3, svr_tr_SCORE1, m=3)
optimal_param3 = samples[np.argmax(svr_SCORE3)]
print(f'optimal_epsilon1: {optimal_param3}')
print(f'optimal r2_score for Mission3: {np.max(svr_SCORE3)}')
plt.savefig(f'Images/Results/svr_mission3_scores.png')

In [ ]:
score_svr = []
print(f'Mission1')
y_pred1, score_out1 = test_model(optim_est1, X1_rfe, y1, testX1_rfe, np.array(testy1))
score_svr.append(score_out1)
print(f'Mission2')
y_pred2, score_out2 = test_model(optim_est2, X2_rfe, y2, testX2_rfe, np.array(testy2))
score_svr.append(score_out2)
print(f'Mission3')
y_pred3, score_out3 = test_model(optim_est3, X3_rfe, y3, testX3_rfe, np.array(testy3))
score_svr.append(score_out3)

In [ ]:
plot_pred_dist(y1, y_pred1, 1)
plt.savefig(f'Images/Results/svr_M1_dist.png')
plot_pred_dist(y2, y_pred2, 2)
plt.savefig(f'Images/Results/svr_M2_dist.png')
plot_pred_dist(y3, y_pred3, 3)
plt.savefig(f'Images/Results/svr_M3_dist.png')

### 3. Ridge Linear Regression 

In [ ]:
lr_SCORE1, lr_SCORE2, lr_SCORE3 = [], [], []
lr_tr_SCORE1, lr_tr_SCORE2, lr_tr_SCORE3 = [], [], []
ESTIMATOR1, ESTIMATOR2, ESTIMATOR3 = [], [], []

samples = np.arange(0.01, 10, 0.1)
for c in samples:
    lr_score1, train_score1, estimator1 = model_selection(Ridge(c), X1_rfe, y1, testX1_rfe, testy1)
    lr_SCORE1.append(lr_score1)
    ESTIMATOR1.append(estimator1)
    lr_tr_SCORE1.append(train_score1)
    lr_score2, train_score2, estimator2 = model_selection(Ridge(c), X2_rfe, y2, testX2_rfe, testy2)
    lr_SCORE2.append(lr_score2)
    ESTIMATOR2.append(estimator2)
    lr_tr_SCORE2.append(train_score2)
    lr_score3, train_score3, estimator3 = model_selection(Ridge(c), X3_rfe, y3, testX3_rfe, testy3)
    lr_SCORE3.append(lr_score3)
    ESTIMATOR3.append(estimator3)
    lr_tr_SCORE3.append(train_score3)

optim_est1 = ESTIMATOR1[np.argmax(lr_SCORE1)]
plot_model_param(samples, lr_SCORE1, lr_tr_SCORE1, m=1)
optimal_param1 = samples[np.argmax(lr_SCORE1)]
print(f'optimal_epsilon1: {optimal_param1}')
print(f'optimal r2_score for Mission1: {np.max(lr_SCORE1)}')
plt.savefig(f'Images/Results/rlr_mission1_scores.png')

optim_est2 = ESTIMATOR2[np.argmax(lr_SCORE2)]
plot_model_param(samples, lr_SCORE2, lr_tr_SCORE2, m=2)
optimal_param2 = samples[np.argmax(lr_SCORE2)]
print(f'optimal_epsilon2: {optimal_param2}')
print(f'optimal r2_score for Mission2: {np.max(lr_SCORE2)}')
plt.savefig(f'Images/Results/rlr_mission2_scores.png')

optim_est3 = ESTIMATOR3[np.argmax(lr_SCORE3)]
plot_model_param(samples, lr_SCORE3, lr_tr_SCORE1, m=3)
optimal_param3 = samples[np.argmax(lr_SCORE3)]
print(f'optimal_epsilon1: {optimal_param3}')
print(f'optimal r2_score for Mission3: {np.max(lr_SCORE3)}')
plt.savefig(f'Images/Results/knn_mission3_scores.png')

In [ ]:
score_rlr = []
print(f'Mission1')
y_pred1, score_out1 = test_model(optim_est1, X1_rfe, y1, testX1_rfe, np.array(testy1))
score_rlr.append(score_out1)
print(f'Mission2')
y_pred2, score_out2 = test_model(optim_est2, X2_rfe, y2, testX2_rfe, np.array(testy2))
score_rlr.append(score_out2)
print(f'Mission3')
y_pred3, score_out3 = test_model(optim_est3, X3_rfe, y3, testX3_rfe, np.array(testy3))
score_rlr.append(score_out3)

In [ ]:
plot_pred_dist(y1, y_pred1, 1)
plt.savefig(f'Images/Results/rlr_M1_dist.png')
plot_pred_dist(y2, y_pred2, 2)
plt.savefig(f'Images/Results/rlr_M2_dist.png')
plot_pred_dist(y3, y_pred3, 3)
plt.savefig(f'Images/Results/rlr_M3_dist.png')

### 4. ANN

In [ ]:
nn_SCORE1, nn_SCORE2, nn_SCORE3 = [], [], []
nn_tr_SCORE1, nn_tr_SCORE2, nn_tr_SCORE3 = [], [], []
ESTIMATOR1, ESTIMATOR2, ESTIMATOR3 = [], [], []

samples = np.arange(100, 120, 1)
for c in samples:
    nn_score1, train_score1, estimator1 = model_selection(MLPRegressor(hidden_layer_sizes=(c,)), X1_rfe, y1, testX1_rfe, testy1)
    nn_SCORE1.append(nn_score1)
    ESTIMATOR1.append(estimator1)
    nn_tr_SCORE1.append(train_score1)
    nn_score2, train_score2, estimator2 = model_selection(MLPRegressor(hidden_layer_sizes=(c,)), X2_rfe, y2, testX2_rfe, testy2)
    nn_SCORE2.append(nn_score2)
    ESTIMATOR2.append(estimator2)
    nn_tr_SCORE2.append(train_score2)
    nn_score3, train_score3, estimator3 = model_selection(MLPRegressor(hidden_layer_sizes=(c,)), X3_rfe, y3, testX3_rfe, testy3)
    nn_SCORE3.append(nn_score3)
    ESTIMATOR3.append(estimator3)
    nn_tr_SCORE3.append(train_score3)

optim_est1 = ESTIMATOR1[np.argmax(nn_SCORE1)]
plot_model_param(samples, nn_SCORE1, nn_tr_SCORE1, m=1)
optimal_param1 = samples[np.argmax(nn_SCORE1)]
print(f'optimal_epsilon1: {optimal_param1}')
print(f'optimal r2_score for Mission1: {np.max(nn_SCORE1)}')
plt.savefig(f'Images/Results/nn_mission1_scores.png')

optim_est2 = ESTIMATOR2[np.argmax(nn_SCORE2)]
plot_model_param(samples, nn_SCORE2, nn_tr_SCORE2, m=2)
optimal_param2 = samples[np.argmax(nn_SCORE2)]
print(f'optimal_epsilon2: {optimal_param2}')
print(f'optimal r2_score for Mission2: {np.max(nn_SCORE2)}')
plt.savefig(f'Images/Results/nn_mission2_scores.png')

optim_est3 = ESTIMATOR3[np.argmax(nn_SCORE3)]
# plot_model_param(samples, lr_SCORE3, nn_tr_SCORE1, m=3)
optimal_param3 = samples[np.argmax(nn_SCORE3)]
print(f'optimal_epsilon1: {optimal_param3}')
print(f'optimal r2_score for Mission3: {np.max(nn_SCORE3)}')
# plt.savefig(f'Images/Results/nn_mission3_scores.png')

In [ ]:
score_nn = []
print(f'Mission1')
y_pred1, score_out1 = test_model(optim_est1, X1_rfe, y1, testX1_rfe, np.array(testy1))
score_nn.append(score_out1)
print(f'Mission2')
y_pred2, score_out2 = test_model(optim_est2, X2_rfe, y2, testX2_rfe, np.array(testy2))
score_nn.append(score_out2)
print(f'Mission3')
y_pred3, score_out3 = test_model(optim_est3, X3_rfe, y3, testX3_rfe, np.array(testy3))
score_nn.append(score_out3)

In [ ]:
plot_pred_dist(y1, y_pred1, 1)
plt.savefig(f'Images/Results/nn_M1_dist.png')
plot_pred_dist(y2, y_pred2, 2)
plt.savefig(f'Images/Results/nn_M2_dist.png')
plot_pred_dist(y3, y_pred3, 3)
plt.savefig(f'Images/Results/nn_M3_dist.png')

# RESULTS

In [ ]:
def plot_model_comp(score1, score2, score3, score4, m):
    plt.figure()
    plt.rcdefaults()
    w = 0.2
    models = ["KNN", "SVR", "Ridge", "NN"]
    ypos1 = np.arange(len(score1))
    ypos2 = [i+w for i in ypos1]
    ypos3 = [i+w for i in ypos2]
    ypos4 = [i+w for i in ypos3]
    plt.bar(ypos1, score1, width=w, label="KNN")
    plt.bar(ypos2, score2, width=w, label="SVR")
    plt.bar(ypos3, score3, width=w, label="Ridge")
    plt.bar(ypos4, score4, width=w, label="NN")
    plt.legend()
    plt.title(m+' for all the Missions')
    plt.savefig('Images/Results/Final_score_plots_' + m + '.png')

In [ ]:
plot_model_comp(score_knn[:][0], score_svr[:][0], score_rlr[:][0], score_nn[:][0], "RMSE")
plot_model_comp(score_knn[:][1], score_svr[:][1], score_knn[:][1], score_nn[:][1], "MAE")
plot_model_comp(score_knn[:][2], score_svr[:][2], score_knn[:][2], score_nn[:][2], "R2")